<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/07_CausalLMFinetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a hacer fine-tuning de un LM causal con [**GPT-2**](https://huggingface.co/docs/transformers/model_doc/gpt2):

* Es un LM (causal) de transformers
* Datos de entrenamiento: _WebText_ (scraping de links que salen de reddit con al menos 3 upvotes)
* Tokenizador: subword tokenization con BPE (Byte Pair Encoding)

Aunque en realidad vamos a usar una versión _destilada_: **distilled-GPT2**.

_Knowledge distillation_ es un proceso que entrena una versión reducida de un modelo más grande al que se intenta imitar, con el objetivo de acelerar el procesamiento y el finetuning en tareas específicas, sacrificando poca performance (ver https://arxiv.org/pdf/1910.01108v4.pdf y https://arxiv.org/pdf/2006.05525.pdf).

-----------------------

Tarea: responder donde dice **PREGUNTA**

## Configuración del entorno

In [ ]:
!pip install -qU torch datasets transformers watermark

In [ ]:
%load_ext watermark

In [ ]:
%watermark -vp torch,transformers,datasets,pandas,numpy

Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU".

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Data

Cargamos [reviews de yelp](https://huggingface.co/datasets/yelp_review_full). Vamos a usar solo algunos ejemplos para trabajar más rápido.

Para cargar un dataset propio ver https://huggingface.co/docs/datasets/loading.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [ ]:
dataset

In [ ]:
print(*dataset["train"].features.items(), sep="\n")

In [ ]:
# 5k train, 2k validation, 5k test
from datasets import DatasetDict

small_dataset = DatasetDict(
    train=dataset["train"].shuffle(seed=33).select(range(0, 5_000)),
    val=dataset["train"].shuffle(seed=33).select(range(10_000, 12_000)),
    test=dataset["test"].shuffle(seed=33).select(range(5_000)),
)

**PREGUNTA 1**: ¿Por qué podríamos necesitar tres sets aún si no tuneamos hiperparámetros?

In [ ]:
print(small_dataset["train"][0]["text"])

In [ ]:
import re

def clean_text(example):
    """Corrige caracteres raros segun la doc de yelp
    """
    texto = re.sub(r'\\n', '\n', example["text"]) # real newlines
    texto = re.sub(r'\\"', '"', texto) # comillas de verdad
    example["text"] = texto
    return example

In [ ]:
small_dataset = small_dataset.map(clean_text)

In [ ]:
print(small_dataset["train"][0]["text"])

## Tokenización y modelo

El max_length admitido por el modelo es 1024 pero esto puede consumir mucha memoria. Entonces vamos a trabajar con un max_length de 128 tokens.

En particular, vamos a partir cada documento en pedazos de 128 tokens. Vamos a tener algunos pedazos con menos de 128 porque hay documentos que no llegan a esta cantidad, y también por los pedazos que queden al final de documentos largos.

Para poder hacer un procesamiento en batches vamos a necesitar _padding_: completar con un token especial hasta llegar al max_length o a la máxima longitud del batch.

Una alternativa es truncar los documentos con más de 128 tokens pero si tenemos muchos documentos largos esto puede implicar tirar mucha información.

Vamos a cargar el tokenizador y los pesos de un modelo pre-entrenado: a esto se le llama **checkpoint**. En este caso, la arquitectura es GPT-2 Distilled, mientras que el checkpoint (los pesos específicos) se llama `distilgpt2`.

Vamos a cargar tokenizer y modelo con `AutoClass`es que permiten cargar checkpoints de cualquier arquitectura rápidamente.

In [ ]:
model_checkpoint = "distilgpt2"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# https://huggingface.co/docs/transformers/main_classes/tokenizer#tokenizer

In [ ]:
tokenizer.model_max_length # Hay solo model_max_length embeddings de posicion

In [ ]:
# context_length = tokenizer.model_max_length
context_length = 128

In [ ]:
# veamos cómo funciona la tokenización en 3 ejemplos
ejemplos = small_dataset["train"][:3]
ejemplos

In [ ]:
outputs_ = tokenizer(
    ejemplos["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True, # tokeniza doc y lo parte en pedazos
    return_length=True, # computa length de cada doc
)

In [ ]:
# como ouput obtenemos token_ids y attention_mask
# por el momento solo vamos a usar token_ids
outputs_

In [ ]:
print(f"Cantidad de chunks: {len(outputs_['input_ids'])}")
print(f"Tokens en cada chunk: {(outputs_['length'])}")
print(f"Mapping chunk-doc: {outputs_['overflow_to_sample_mapping']}")

In [ ]:
# con tokenize() obtenemos la separación en subwords
tokens_ = tokenizer.tokenize(ejemplos["text"][0])
print(tokens_)

In [ ]:
# el tokenizer de gpt2 trata a los espacios como parte de las palabras,
# entonces codifica distinto a las palabras en el medio vs el principio de la
# secuencia
# https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2Tokenizer

print(tokenizer.tokenize("Love this place"))
print(tokenizer("Love this place")['input_ids'])
print(tokenizer.tokenize(" Love this place"))
print(tokenizer(" Love this place")['input_ids'])

In [ ]:
def tokenize_fn(example):
    """Tokeniza `text` de examples de un dataset.
    Returns only input_ids.
    """
    outputs = tokenizer(
        example["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    return {"input_ids": outputs["input_ids"]}

In [ ]:
# Aplicamos la tokenizacion en batches y 4 procesos para acelerar la corrida
    # descartamos el resto de columnas
tokenized_dataset = small_dataset.map(
    tokenize_fn, batched=True, num_proc=4,
    remove_columns=small_dataset["train"].column_names)

# NOTE: si queremos conservar mas columnas, tenemos que generar la misma
# cantidad de datos que en el output (esta tokenizacion genera mas samples
# que la cantidad inicial de examples)

In [ ]:
small_dataset

In [ ]:
tokenized_dataset

**PREGUNTA 2**: ¿qué representa cada _row_ de tokenized_dataset?

In [ ]:
# Cargamos el modelo
    # Usamos el EOS token as PAD token to avoid warnings (GPT2 does not have a PAD token)
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, pad_token_id=tokenizer.eos_token_id)

In [ ]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**2:.1f}M parameters")
# numel: number of elements in tensor

# gpt3 tiene 175B params, gpt4 tiene 1T...

In [ ]:
print(model)

## Entrenamiento

Un "collator" es una función que forma batches de datos.

Vamos a usar un "collator" que arma batches de ejemplos con padding. `DataCollatorForLanguageModeling` está diseñado específicamente para language models.

En particular se encarga de:

* armar los targets del modelo (los tokens desplazados) _on the fly_ durante el entrenamiento sin duplicar los input_ids.
* Agregar padding donde corresponda

Usamos `mlm=False` para usar **Causal Language Modeling** en lugar de Masked Language Modeling.

Podemos loguear métricas durante el entrenamiento con tensorboard, wandb, etc.

In [ ]:
# el padding se hace con el EOS token
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
# vemos un ejemplo con un batch de 3 docs
out = data_collator([tokenized_dataset["train"][i] for i in range(3)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

In [ ]:
# hay padding:
out["input_ids"][1]

In [ ]:
# attention mask para no hacer attention sobre pad_tokens:
out["attention_mask"][1]

In [ ]:
# usamos solo el nombre del modelo para el nuevo nombre (no el usuario)
pretrained_model_name = model_checkpoint.split("/")[-1]
finetuned_model_name = f"{pretrained_model_name}-finetuned-yelp"
print(finetuned_model_name)

Si vamos a usar wandb, copiamos API key de https://wandb.ai/authorize

In [ ]:
#!wandb login

In [ ]:
#os.environ["WANDB_PROJECT"] = project_name

In [ ]:
# definimos los parametros del entrenamiento
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    finetuned_model_name,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-4,
    weight_decay=0.1, # forma de regularizacion (restringe el tamaño de updates de SGD)
    warmup_ratio=0.1, # # warmup evita divergencia de loss en primeros steps (10%)
    lr_scheduler_type="cosine",
    do_eval=True, # eval en validation set
    gradient_accumulation_steps=1, # acumula gradientes por N steps --> update cada N*32 samples
    # sirve cuando batches grandes no entran en memoria y tenemos muchos samples
    eval_strategy="steps", # eval en validation set
    eval_steps=50,
    save_strategy="steps",
    load_best_model_at_end=True, # conserva mejor modelo segun eval loss
    save_total_limit=2, # save max 2 models including best one
    save_steps=50, # checkpoint model every N steps
    logging_dir='./logs', # logging
    logging_strategy="steps",
    logging_steps=1,
    fp16=True, # float16 en training (only on CUDA)
    push_to_hub=False,
#    report_to="wandb",  # enable logging to W&B
   report_to="none",
    save_safetensors=False # por un bug
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"], #.select(range(0, 128)),
    eval_dataset=tokenized_dataset["val"], #.select(range(0, 128)),
)

**PREGUNTA 3**: ¿qué es el parámetro de learning_rate?

In [ ]:
#!rm -rf ./logs # para wandb/tensorboard

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir logs

# para wandb/tensorboard

In [ ]:
# Entrenamos!
train_output = trainer.train()

In [ ]:
# para guardar el modelo:
trainer.save_model()

## Evaluation

In [ ]:
train_output

In [ ]:
# volvemos a calcular loss en train porque train_output.training_loss
# se calcula con criterio distinto a trainer.evaluate()
train_results = trainer.evaluate(tokenized_dataset["train"])
val_results = trainer.evaluate()
test_results = trainer.evaluate(tokenized_dataset["test"])

In [ ]:
train_results

In [ ]:
val_results

In [ ]:
import numpy as np

print("Perplexity:")
print(f"Train: {np.exp(train_results['eval_loss']):.2f}")
print(f"Validation: {np.exp(val_results['eval_loss']):.2f}")
print(f"Test: {np.exp(test_results['eval_loss']):.2f}")

In [ ]:
# comparamos con el GPT2 no fine-tuneado
    # un poco hackoso, instanciamos un trainer pero no vamos a entrenar
    # es solo para replicar exactamente la evaluacion anterior, sería
    # mejor armar una funcion adhoc
model_original = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, pad_token_id=tokenizer.eos_token_id)
trainer_aux = Trainer(
    model=model_original,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"], #.select(range(0, 128)),
    eval_dataset=tokenized_dataset["test"], #.select(range(0, 128)),
)

In [ ]:
test_results_original = trainer_aux.evaluate(tokenized_dataset["test"])

In [ ]:
print("Perplexity (no fine-tuning):")
print(f"Test: {np.exp(test_results_original['eval_loss']):.2f}")

**PREGUNTA 4** ¿por qué la versión fine-tuned tiene menos perplexity que sin fine-tuning?

### Text generation

In [ ]:
import torch

device = f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"

In [ ]:
def generate(
    prompt=None, max_length=100, greedy=True, model=model, tokenizer=tokenizer, device=device
):
    """Generar texto con sampling (greedy=False) o greedy search (greedy=True)

    prompt=None stands for beggining of sequence.

    NOTE si bien parece que GPT2 puede generar a partir de BOS token, la
    documentacion es poco clara. Ademas hicimos nuestro finetuning sin BOS token.
    Entonces solo vamos a usar la funcion pasandole un contexto.

    Ver:
    https://github.com/huggingface/transformers/issues/3311#issuecomment-601264426
    https://github.com/openai/gpt-2/blob/a74da5d99abaaba920de8131d64da2862a8f213b/src/generate_unconditional_samples.py#L60
    """
    do_sample = False if greedy else True
    # model.eval() to set dropout and batch normalization layers to evaluation mode before running inference
    model.eval()
    with torch.inference_mode():
        if prompt:
            input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
            outputs = model.generate(input_ids, do_sample=do_sample, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
        else:
            outputs = model.generate(do_sample=do_sample, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    # pad_token_id=tokenizer.eos_token_id to suppress warning
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
res_ = generate('I loved "El Topo" because')
print(res_[0])

**PREGUNTA 5**: ¿con un mismo prompt vamos a obtener siempre la misma generación?

In [ ]:
torch.manual_seed(33)
res_ = generate('I loved "El Topo" because', greedy=False)
print(res_[0])

In [ ]:
torch.manual_seed(0)
res_ = generate('I loved "El Topo" because', greedy=False)
print(res_[0])

In [ ]:
torch.manual_seed(33)
res_ = generate('I loved "El Topo" because', greedy=False, model=model_original)
print(res_[0])

**PREGUNTA 6** ¿por qué el formato y contenido del texto generado con el modelo sin fine-tuning es tan distinto al modelo fine-tuned?

In [ ]:
torch.manual_seed(23)
res_ = generate('I hated the cake from "El Topo" because', greedy=False)
print(res_[0])

In [ ]:
generate('It was the worst day ever because', greedy=False)

## Referencias

* [Causal LM from sratch](https://huggingface.co/course/chapter7/6?#training-a-causal-language-model-from-scratch)

* [LM finetuning](https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/language_modeling.ipynb)

* [Customized training](https://huggingface.co/course/chapter3/4#a-full-training)

* [Text generation](https://github.com/huggingface/blog/blob/main/notebooks/02_how_to_generate.ipynb)

* [Scripts para entrenar y finetunear modelos](https://github.com/huggingface/transformers/tree/main/examples/pytorch)

* [Sobre GPT-2](https://huggingface.co/gpt2)

* [Autoclasses](https://huggingface.co/docs/transformers/autoclass_tutorial)

* [Hugging Face + wandb](https://docs.wandb.ai/guides/integrations/huggingface) (no logré hacerlo andar bien en colab 😞)

* [Howard & Gugger (2020) - Deep learning for coders with fastai and PyTorch](https://dl.ebooksworld.ir/books/Deep.Learning.for.Coders.with.fastai.and.PyTorch.Howard.Gugger.OReilly.9781492045526.EBooksWorld.ir.pdf) -- temas generales de fine-tuning y DL